In [1]:
# imports
import wget
import zipfile
import os
import pandas as pd
import shutil
import collections
import pickle 

In [2]:
# globals
dataset_url = 'https://hosted-datasets.gbif.org/datasets/ipni.zip'
dataset_dir = '/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data/corpus_taxon/'
output_dir = '/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-angeline1/ml-herbarium/corpus/corpus_taxon/'
csv_dir = 'ipni2/ipni.csv'
csv = 'ipni.csv'
zip_file = 'ipni.zip'
corpus_taxon = 'corpus_taxon.txt'
duplicates = 'duplicates_taxon.txt'

# Download Dataset

In [3]:
# FIXME: need to update the new zip file
# if os.path.exists(dataset_dir + zip_file):
#     os.remove(dataset_dir + zip_file)
# wget.download(dataset_url, out=dataset_dir + zip_file)

# Extract .csv from .zip

In [3]:
if os.path.exists(dataset_dir + 'ipni2'):
    shutil.rmtree(dataset_dir + 'ipni2')
with zipfile.ZipFile(dataset_dir + zip_file,'r') as zip_ref:
    zip_ref.extract(member=csv_dir, path=dataset_dir)
zf = zipfile.ZipFile(dataset_dir + zip_file)
df = pd.read_csv(csv_dir, header=None)

In [4]:
# zf.infolist()

In [5]:
# print out all the columns of first row to identify which column taxon is located
colnames = []
for col in df.columns:
    colnames.append(col)
print(colnames)
df.head()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


,0,1,2,3,4,5,6,7,8,9,10,11
0,urn:lsid:ipni.org:names:327752-1,Desmia conferta,D.Don,spec.,Ericaceae,NaN,urn:lsid:ipni.org:names:14701-1,NaN,1834.0,Edinburgh New Philos. J. 17: 153. 1834 [Jul 1...,unknown,https://www.ipni.org/n/urn:lsid:ipni.org:names...
1,urn:lsid:ipni.org:names:327752-2,trib. Paniceae,R.Br. in Flinders,trib.,Poaceae,urn:lsid:ipni.org:names:321663-2,urn:lsid:ipni.org:names:30000032-2,urn:lsid:ipni.org:publications:9050-2,1814.0,Voy. Terra Austral. 2: 582. 1814 [19 Jul 1814],tax. nov.,https://www.ipni.org/n/urn:lsid:ipni.org:names...
2,urn:lsid:ipni.org:names:327753-1,Desmia polifolia,D.Don,spec.,Ericaceae,NaN,urn:lsid:ipni.org:names:14701-1,NaN,1834.0,Edinburgh New Philos. J. 17: 153. 1834 [Jul 1...,unknown,https://www.ipni.org/n/urn:lsid:ipni.org:names...
3,urn:lsid:ipni.org:names:327753-2,Chamaesyce,Gray,gen.,Euphorbiaceae,NaN,urn:lsid:ipni.org:names:30000064-2,urn:lsid:ipni.org:publications:4868-2,1821.0,Nat. Arr. Brit. Pl. 2: 260. 1821 [1 Nov 1821],tax. nov.,https://www.ipni.org/n/urn:lsid:ipni.org:names...
4,urn:lsid:ipni.org:names:327754-1,Desmogyne angustifolia,Knagg,spec.,Ericaceae,NaN,urn:lsid:ipni.org:names:50325670-1,urn:lsid:ipni.org:publications:758-2,1923.0,Notes Roy. Bot. Gard. Edinburgh 14: 73. 1923,unknown,https://www.ipni.org/n/urn:lsid:ipni.org:names...


# Export Taxon Name to corpus-taxon.txt

In [6]:
# taxon is located in the second column 
print (df[1]) 

0                 Desmia conferta
1                  trib. Paniceae
2                Desmia polifolia
3                      Chamaesyce
4          Desmogyne angustifolia
                    ...          
1352345                Cyclaminos
1352346       Bigelowia nuttallii
1352347        subtrib. Zoysiinae
1352348           Desmia aequalis
1352349             trib. Oryzeae
Name: 1, Length: 1352350, dtype: object


In [7]:
# we need to take care of the duplicates in the dataset 
list_taxon = df[1].to_list()
list_taxon_no_dup = list(set(list_taxon))

print(list_taxon[:10])
print(len(list_taxon))
print(len(list_taxon_no_dup))

['Desmia conferta', 'trib. Paniceae', 'Desmia polifolia', 'Chamaesyce', 'Desmogyne angustifolia', 'Synadenium', 'Blakea subpanduriformis', 'Utricularia racemosa', 'Topobea adscendens', 'Topobea bullata']
1352350
1222306


In [8]:
dup = [item for item, count in collections.Counter(list_taxon).items() if count > 1]
print(len(dup))
print(len(set(dup)))
# print(dup)

105129
105129


In [9]:
# print out the duplicates on duplicates_taxon.txt
dup = [item for item, count in collections.Counter(list_taxon).items() if count > 1]
with open(output_dir + duplicates, "w") as output_file:
    for i in range(len(dup)):
        output_file.write(dup[i] + '\n')
    output_file.close()

In [10]:
with open(output_dir + duplicates, 'r') as fp:
    x = len(fp.readlines())
    print('Total lines:', x) 

Total lines: 105129


In [11]:
# print out the taxon names without duplicates 
with open(output_dir + corpus_taxon, "w") as output_file:
    with open(output_dir + "single.txt", "w") as file:
        dict_possible_species = {}
        dict_possible_genus = {}
        for i in range(len(list_taxon_no_dup)):
            # get rid of single-word taxons
            if ' ' in list_taxon_no_dup[i]:
                output_file.write(list_taxon_no_dup[i] + '\n')
                genus = list_taxon_no_dup[i].split()[0].lower()
                # FIXME: just take the first word of the whole species name for easier match 
                species = "".join(list_taxon_no_dup[i].split()[1:]).lower()
                # FIXME: need to account for the remaining species [2:]
                # generate a dictionary of possible species for each genus
                if genus not in dict_possible_species:
                    dict_possible_species[genus] = [species]
                else:
                    # do not include duplicated species 
                    if species not in dict_possible_species[genus]:
                        dict_possible_species[genus] += [species]
                # generate a dictionary of possible genera for each species
                if species not in dict_possible_genus:
                    dict_possible_genus[species] = [genus]
                else:
                    # do not include duplicated geneus
                    if genus not in dict_possible_genus[species]:
                        dict_possible_genus[species] += [genus]
            else:
                file.write(list_taxon_no_dup[i] + '\n')
        with open(dataset_dir + 'output/possible_species.pkl', 'wb') as f:
            pickle.dump(dict_possible_species, f) 
        with open(dataset_dir + 'output/possible_genus.pkl', 'wb') as filee:
            pickle.dump(dict_possible_genus, filee) 
            
file.close()
output_file.close()

In [12]:
with open(output_dir + corpus_taxon, 'r') as fp:
    x = len(fp.readlines())
    print('Total lines:', x)

Total lines: 1181366


In [13]:
with open(dataset_dir + 'output/possible_species.pkl', 'rb') as f:
    dict = pickle.load(f)
with open(dataset_dir + 'output/possible_genus.pkl', 'rb') as ff:
    dictt = pickle.load(ff)

In [14]:
print(len(dict))
print(len(dictt))

34829
381368


In [15]:
dictt["boala"]

['diospyros']

In [16]:
dict['rea']

['pinnata',
 'micrantha',
 'macrantha',
 'marginata',
 'berteroana',
 'longifolia',
 'lacerata',
 'neriifolia',
 'pruinata',
 'leucantha',
 'pinnatavar.insignis',
 'mollis']

In [17]:
from difflib import get_close_matches
test = get_close_matches("lungicadatus", dictt, n=1, cutoff=0.8)
test

['longicaudatus']

In [18]:
summ1 = 0
summ2 = 0
for key, value in dict.items():
    va = sum(1 for v in value if v)
    summ1 += va
print("total number of species:", summ1)
for key, value in dictt.items():
    va = sum(1 for v in value if v)
    summ2 += va
print("total number of genus:", summ2)


total number of species: 1181365
total number of genus: 1181365
